# Experiment Setup and Validation Notebook

This notebook walks you through setting up a JSPsych experiment with WAVE backend integration. It will:

1. **Setup Phase**: Guide you through installing dependencies and testing your experiment locally
2. **Schema Definition**: Help you define the experiment data schema that matches your experiment's output
3. **Backend Integration**: Create and test experiment types in the WAVE backend
4. **Validation**: Run a complete test cycle to ensure data logging works correctly
5. **Production Setup**: Create your final experiment for real data collection

**Prerequisites**: This notebook assumes you have already (mostly) set up your experiment code and WAVE client integration in `src/js/integrations/wave-client.js`. 
Also, copy `tools/.env.example` and rename it `tools/.env`, with API key information filled in

**Important**: Only share EXPERIMENTEE-level API keys with participants. Keep RESEARCHER-level keys secure and private.

## Setup Instructions

### 1. Install uv Package Manager

**SKIP THIS STEP IF YOU'VE ALREADY INSTALLED UV FOLLOWING THE README STEPS**

First, install [uv](https://docs.astral.sh/uv/) - a fast Python package manager and project manager:

```bash
# On macOS/Linux:
curl -LsSf https://astral.sh/uv/install.sh | sh

# On Windows:
powershell -ExecutionPolicy ByPass -c "irm https://astral.sh/uv/install.ps1 | iex"
```

### 2. Install Project Dependencies

**SKIP THIS STEP IF YOU'VE ALREADY INSTALLED UV FOLLOWING THE README STEPS**

In the root directory of this project, install the Python dependencies:

```bash
uv sync
```

This will install all dependencies defined in `pyproject.toml`, which specifies the packages needed for experiment setup, data validation, and WAVE backend integration.

### 3. Understanding UV Virtual Environment

UV automatically creates and manages a virtual environment in the `.venv` folder. This keeps your project dependencies isolated from your system Python installation.

**Important**: If you encounter any package conflicts or want a completely clean slate, simply delete the `.venv` folder and `uv.lock` file and run `uv sync` again to recreate it from scratch.

### 4. Running This Notebook

You have multiple options for running this notebook:

**Option A: VS Code Jupyter Extension**
1. Install the [Jupyter VSCode extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)
2. Open this notebook file in VS Code
3. When prompted to select a kernel, choose the Python interpreter from the `.venv` folder:
   - Click "Select Kernel" in the top-right of the notebook
   - Choose "Python Environments..."
   - Select the interpreter at `.venv/bin/python` (or `.venv/Scripts/python.exe` on Windows)
4. VS Code will use this virtual environment for all notebook cells


**Option B: Jupyter Lab/Notebook**
```bash
uv tool run jupyter lab
```
This will install dependencies and launch a Jupyter Lab instance in your browser. Navigate to `setup_experiment.ipynb` in the Jupyter file explorer and run this notebook from there


**Option C: PyCharm IDE**
1. Make sure your uv dependencies were installed following the instructions [here](https://www.jetbrains.com/help/pycharm/uv.html)
2. Open `setup_experiment.ipynb` in your IDE


### 5. Verify Installation

Run all of the input cells below, if they don't import the modules correctly, then something went wrong. Try the install procedures again

In [1]:
from wave_client import (
    WaveClient,
)  # <-- If this import doesn't work, then something went wrong with the Wave Client install!!!

In [2]:
from utils import (
    load_environment_variables,
    start_local_server,
    get_existing_experiment_types,
    check_naming_conflicts,
    create_experiment_type,
    generate_test_identifiers,
    get_existing_tags,
    check_tags_to_create,
    create_missing_tags,
    create_experiment,
    create_experiment_url,
    get_experiment_data,
    get_user_confirmation,
    print_schema_info,
)
from wave_client.models.base import ExperimentTypeCreate
import pandas as pd
import sys
import webbrowser

In [3]:
import warnings

warnings.filterwarnings("ignore", message="To exit: use 'exit', 'quit', or Ctrl-D.")

## Phase 0: Setting up WAVE credentials

### Important: Environment Setup

**Before proceeding, ensure you have:**
1. **Copied `tools/.env.example` to `tools/.env`**
2. **Filled in your WAVE API keys in the `.env` file:**
   - `RESEARCHER_API_KEY` - Your researcher-level key (full permissions)
   - `EXPERIMENTEE_API_KEY` - Your participant-level key (data logging only)  
   - `WAVE_BACKEND_URL` - The WAVE backend URL (must match the variable `waveBackendUrl` in file `src/js/core/params.js`)

⚠️ **Security Note**: Never share your RESEARCHER API key. Only distribute EXPERIMENTEE keys to participants.



In [4]:
# Load environment variables and connect to WAVE backend
RESEARCHER_API_KEY, EXPERIMENTEE_API_KEY, WAVE_BACKEND_URL = load_environment_variables()

print(f"🔍 Connecting to WAVE backend...")
print(f"📡 Backend URL: {WAVE_BACKEND_URL}")
print(f"🔑 Using Researcher API key ending in: ...{RESEARCHER_API_KEY[-4:]}")

# Get existing experiment types
existing_experiment_types = await get_existing_experiment_types(
    RESEARCHER_API_KEY, WAVE_BACKEND_URL
)
print("✅ Ready to set up our experiment")

Present working directory: I:\My Drive\Experiments\Code\JS-Williams\experiment-template\tools
Loading environment variables from I:\My Drive\Experiments\Code\JS-Williams\experiment-template\tools/.env
🔍 Connecting to WAVE backend...
📡 Backend URL: https://wave-backend-production-8781.up.railway.app
🔑 Using Researcher API key ending in: ...SYuP

✅ Connected successfully!
📊 Found 1 existing experiment types:
  - jspsych_color_circles_demo (table: jspsych_circles_data) [ID: 1]
✅ Ready to set up our experiment


## Phase 1: Local Experiment Testing

### Test Your Experiment Locally

Now let's open your experiment in a web browser to test it locally (without WAVE logging).

In [5]:
PORT = 8080  # We will first run locally on localhost:<PORT>

## Make sure to close any previous local test tabs. Otherwise you'll have to restart the notebook kernel and run everything from the beginning of this notebook again.

# Start local server and open experiment
experiment_url = start_local_server(
    port=PORT, experiment_root="../"
)  # Modify this path if your experiment root (index.html) is located elsewhere

print(
    f"\n💡 Note: To stop server, restart the notebook kernel or run 'Kernel > Restart' in Jupyter"
)

Starting HTTP server at I:\My Drive\Experiments\Code\JS-Williams\experiment-template
✅ Experiment opened in browser
✅ HTTP server running on localhost:8080

💡 Note: To stop server, restart the notebook kernel or run 'Kernel > Restart' in Jupyter


### Next Steps

1. Do a complete runthrough of the entire experiment
2. Check browser console for any errors
3. At the end, verify data appears via `jsPsych.data.displayData()` or console output
4. Return to this notebook when done testing, and proceed with running the next block.
5. Server will keep running until you restart this notebook kernel

In [6]:
# Local experiment validation check
print("🔍 Local Experiment Validation")
print("Please complete the following validation steps:")
print("1. Did you successfully complete the entire experiment?")
print("2. Check the browser console log OR check results with jsPsych.data.displayData()")
print("3. Verify that experiment data was GENERATED (not logged to WAVE yet)")

get_user_confirmation(
    "\nDid the experiment run locally successfully and generate data?",
    "Please fix any issues with your local experiment before continuing.\n"
    + "💡 Common issues:\n"
    + "   - Missing stimulus files\n"
    + "   - JavaScript errors in console\n"
    + "   - Experiment not completing properly\n"
    + "   - No data being generated",
)

print("✅ Great! Your experiment generates data locally.")
print("🔧 Next, we'll set up WAVE backend integration...")
print("\n💡 Note: You can keep the local server running for now.")
print("   We'll use it again later with WAVE integration parameters.")

🔍 Local Experiment Validation
Please complete the following validation steps:
1. Did you successfully complete the entire experiment?
2. Check the browser console log OR check results with jsPsych.data.displayData()
3. Verify that experiment data was GENERATED (not logged to WAVE yet)
Please enter your feedback to continue.


127.0.0.1 - - [01/Jan/2026 01:24:19] "GET / HTTP/1.1" 304 -
127.0.0.1 - - [01/Jan/2026 01:24:19] "GET /src/js/utils/plugin-html-slider-response-resizing.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Jan/2026 01:24:19] "GET /src/js/core/params.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Jan/2026 01:24:19] "GET /src/js/core/instructions.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Jan/2026 01:24:19] "GET /src/js/core/trial.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Jan/2026 01:24:19] "GET /src/js/integrations/wave-client.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Jan/2026 01:24:19] "GET /src/js/core/timeline.js HTTP/1.1" 304 -


✅ Great! Your experiment generates data locally.
🔧 Next, we'll set up WAVE backend integration...

💡 Note: You can keep the local server running for now.
   We'll use it again later with WAVE integration parameters.


## Phase 2: Logging data to the WAVE backend

### Defining Experiment Data Schema

This step is absolutely **critical, be very careful!**

What is happening here: Basically, we need to tell the server what data columns it is about to receive from the experiment (so that it will actually save the data to the server).
To define the data that gets LOGGED to the WAVE backend, open up the file `src/js/integrations/wave-client.js`, and go to the `processTrialData(data)` function (~lines 95-126). Update those lines with the columns you need (based on the data printout of your experiment test run that you just did), then copy-paste it in the "Copy of Current Schema Fields" box below. This "Copy of Current Schema Fields box" is not actually executable code, but it is important for the next step to come.

**Copy of Current Schema Fields from wave-client.js** (Remember that you may need to modify these to match your experiment, and as stated above, you should modify it in the wave-client.js file, then copy-paste that here):
```javascript
const waveData = {
    trial_number: data.trial_index,
    trial_type: data.trial_type,
    trial_category: data.trial_category,
    stimulus: data.stimulus,
    response: data.response,
    response_time: data.rt,
    accuracy: data.thisAcc,
    correct_response: data.correct_response,
    stimulus_duration: data.trial_duration,
    time_elapsed: data.time_elapsed,
    timestamp: data.timestamp,
    user_agent: data.user_agent
};
```
**Notes:**
- The following function processes JSPsych trial data and extracts specific fields for logging, so we need to make what is written above match what is written in the next box
- Currently configured to log data from trials with `trial_category` containing 'expt'
- The extracted `waveData` object (lines 118-131) defines exactly what gets sent to WAVE

⚠️ **IMPORTANT**: The experiment schema in the WAVE backend (that we will define) must exactly match the fields defined in `processTrialData`. If there's a mismatch, data logging will fail.

In [7]:
# Define experiment schema and check for naming conflicts

# Experiment type details - MODIFY THESE EACH TIME YOU NEED A DIFFERENT EXPERIMENT TYPE
# (I.E., A NEW SET OF COLUMNS FOR YOUR DATA LOGGING)
#
# If you define a new experiment type name, you need to also define a new table name
# (Do not change one without also changing the other)
experiment_type_name = "jspsych_color_circles_demo"
table_name = "jspsych_circles_data"  # lowercase, underscores only

# Now here is where you want each entry's key to match whatever is in the box above. We are trying to make the schema definition here match processTrialData in wave-client.js.
experiment_schema = {
    "trial_number": "INTEGER",
    "trial_type": "STRING",
    "trial_category": "STRING",
    "stimulus": "STRING",
    "response": "STRING",
    "response_time": "FLOAT",  # No longer converted from milliseconds to seconds in processTrialData
    "accuracy": "FLOAT",
    "correct_response": "STRING",
    "stimulus_duration": "INTEGER",
    "time_elapsed": "INTEGER",
    "timestamp": "STRING",
    "user_agent": "STRING",
}

print(f"Proposed Experiment Type: {experiment_type_name}")
print(f"Proposed Table Name: {table_name}")
print("\n📊 Schema Fields:")
for field, field_type in experiment_schema.items():
    print(f"  {field}: {field_type}")

# Check for naming conflicts with existing experiment types
skip_experiment_type_creation, existing_experiment_type_id = check_naming_conflicts(
    experiment_type_name, table_name, existing_experiment_types
)

print(f"\n💡 Total fields: {len(experiment_schema)}")

# Confirm schema is correct
get_user_confirmation(
    "\nDoes this schema match your processTrialData function in wave-client.js?",
    "Please modify the experiment_schema dictionary above to match your processTrialData function.\n"
    + "💡 Look at src/js/integrations/wave-client.js lines 95-126",
)

print("✅ Schema confirmed!")
print(f"🎯 Skip experiment type creation: {skip_experiment_type_creation}")
print("✅ Ready for backend setup")

Proposed Experiment Type: jspsych_color_circles_demo
Proposed Table Name: jspsych_circles_data

📊 Schema Fields:
  trial_number: INTEGER
  trial_type: STRING
  trial_category: STRING
  stimulus: STRING
  response: STRING
  response_time: FLOAT
  accuracy: FLOAT
  correct_response: STRING
  stimulus_duration: INTEGER
  time_elapsed: INTEGER
  timestamp: STRING
  user_agent: STRING

⚠️  Experiment type 'jspsych_color_circles_demo' already exists!
✅ Will use existing experiment type (ID: 1)

💡 Total fields: 12
Please enter your feedback to continue.
✅ Schema confirmed!
🎯 Skip experiment type creation: True
✅ Ready for validation and backend setup


In [8]:
print_schema_info()

### Schema Validation Rules

**Reserved Column Names** (cannot be used in your schema):
- `id` - Auto-generated primary key
- `experiment_uuid` - Links data to experiment
- `participant_id` - Participant identifier (added automatically)
- `created_at` - Timestamp when data was created
- `updated_at` - Timestamp when data was last modified

**Supported Data Types**:
- `INTEGER` - Whole numbers (e.g., trial numbers, counts)
- `FLOAT` - Decimal numbers (e.g., reaction times, scores)
- `STRING` - Text up to 255 characters (e.g., responses, stimulus names)
- `TEXT` - Longer text content (unlimited length)
- `BOOLEAN` - True/false values (e.g., accuracy, conditions)
- `DATETIME` - Date and time stamps
- `JSON` - Complex structured data

**Important Notes**:
- STRING fields are capped at 255 characters - use TEXT for longer content
- Column names are case-sensitive and should match exactly
- Field names cannot conflict with reserved names above


In [9]:
# Create the pydantic model with our schema
experiment_type_data = ExperimentTypeCreate(
    name=experiment_type_name,
    table_name=table_name,
    schema_definition=experiment_schema,
    description=f"JSPsych experiment data schema for {experiment_type_name}",
)


🔍 Validating Experiment Schema
✅ Schema validation passed!
✅ Experiment Type: jspsych_color_circles_demo
✅ Table Name: jspsych_circles_data
✅ Fields: 12
✅ No conflicts with reserved names
✅ All data types are supported

🎯 Schema is ready for WAVE backend!


In [10]:
# Create or use existing experiment type in WAVE backend

if skip_experiment_type_creation:
    exp_type_id = existing_experiment_type_id
    print(f"🔄 Using existing experiment type: {experiment_type_name}")
    print(f"📊 Experiment Type ID: {exp_type_id}")
    print("⏭️  Skipping experiment type creation")

else:
    print("🚀 Creating new experiment type in WAVE backend...")

    # Create the pydantic model with our schema
    experiment_type_data = ExperimentTypeCreate(
        name=experiment_type_name,
        table_name=table_name,
        schema_definition=experiment_schema,
        description=f"JSPsych experiment data schema for {experiment_type_name}",
    )

    print("✅ Schema validation passed!")

    # Create the experiment type
    try:
        created_experiment_type = await create_experiment_type(
            experiment_type_data, RESEARCHER_API_KEY, WAVE_BACKEND_URL
        )
        exp_type_id = created_experiment_type["id"]

        print(f"✅ Created experiment type: {created_experiment_type['name']}")
        print(f"📊 Experiment Type ID: {exp_type_id}")
        print(f"🗄️  Database Table: {created_experiment_type['table_name']}")
        print(f"📝 Description: {created_experiment_type['description']}")
        print(f"📊 Schema Fields: {len(created_experiment_type['schema_definition'])}")
        print(f"🕒 Created: {created_experiment_type['created_at']}")

    except Exception as e:
        print(f"❌ Failed to create experiment type: {e}")
        print("💡 Common issues:")
        print("   - API key lacks RESEARCHER permissions")
        print("   - Network connectivity problems")
        print("   - Conflicting experiment type name or table name")
        print("   - Invalid schema definition")
        sys.exit("Fix the issue and re-run this cell")

print(f"\n🎯 Experiment Type ID for next steps: {exp_type_id}")
print("✅ Ready to create test experiment!")

🔄 Using existing experiment type: jspsych_color_circles_demo
📊 Experiment Type ID: 1
⏭️  Skipping experiment type creation

🎯 Experiment Type ID for next steps: 1
✅ Ready to create test experiment!


In [11]:
# Generate unique test experiment name and static experimentee ID
test_experiment_name, test_experimentee_id = generate_test_identifiers(experiment_type_name)

print("🆔 Test Identifiers Generated")
print(f"📝 Test Experiment Name: {test_experiment_name}")
print(f"👤 Test Experimentee ID: {test_experimentee_id}")

print(f"\n💡 The test experiment will be created with:")
print(f"   - Name: {test_experiment_name}")
print(f"   - Type ID: {exp_type_id}")
print(f"   - Participant: {test_experimentee_id}")

print(f"\n✅ Identifiers ready for test experiment creation")

🆔 Test Identifiers Generated
📝 Test Experiment Name: TEST_jspsych_color_circles_demo_20260101_012427
👤 Test Experimentee ID: test_participant_8182729c

💡 The test experiment will be created with:
   - Name: TEST_jspsych_color_circles_demo_20260101_012427
   - Type ID: 1
   - Participant: test_participant_8182729c

✅ Identifiers ready for test experiment creation


In [12]:
# Define experiment tags and check for existence

print("🏷️  Experiment Tags Setup")
print("Tags help categorize and organize your experiments in the WAVE system")

# Define tags for the test experiment - MODIFY THESE AS NEEDED
# Tag names should be either camelCase or lower_case_with_underscores
experiment_tags = [
    {"name": "test", "description": "Test experiments for validation purposes"}, # DO NOT DELETE THIS TAG
]

print(f"\n📋 Proposed experiment tags:")
for tag in experiment_tags:
    print(f"  - {tag['name']}: {tag['description']}")

# Get existing tags from WAVE backend
print(f"\n🔍 Checking existing tags in WAVE backend...")

existing_tags = await get_existing_tags(RESEARCHER_API_KEY, WAVE_BACKEND_URL)

print(f"✅ Found {len(existing_tags)} existing tags:")
if existing_tags:
    for tag in existing_tags:
        print(f"  - {tag['name']}: {tag['description']} [ID: {tag['id']}]")
else:
    print("  (No existing tags found)")

# Check which tags need to be created
tags_to_create = check_tags_to_create(experiment_tags, existing_tags)

if tags_to_create:
    print(f"\n📝 Need to create {len(tags_to_create)} new tags:")
    for tag in tags_to_create:
        print(f"  - {tag['name']}")
else:
    print(f"\n✅ All required tags already exist!")

# Confirm tags are correct
get_user_confirmation(
    "\nAre these tags appropriate for your experiment? (check the output of this cell of code)",
    "Please modify the experiment_tags list above and re-run this cell",
)

print("✅ Tags confirmed!")
tag_names_for_experiment = [tag["name"] for tag in experiment_tags]
print(f"🎯 Tags for experiment: {', '.join(tag_names_for_experiment)}")

🏷️  Experiment Tags Setup
Tags help categorize and organize your experiments in the WAVE system

📋 Proposed experiment tags:
  - test: Test experiments for validation purposes

🔍 Checking existing tags in WAVE backend...
✅ Found 2 existing tags:
  - test: Test experiments for validation purposes [ID: 1]
  - color_react: Reaction time experiments with colors [ID: 2]

✅ All required tags already exist!
Please enter your feedback to continue.
✅ Tags confirmed!
🎯 Tags for experiment: test


In [13]:
# Create missing tags in WAVE backend

if tags_to_create:
    print(f"🚀 Creating {len(tags_to_create)} missing tags...")

    try:
        new_tags = await create_missing_tags(tags_to_create, RESEARCHER_API_KEY, WAVE_BACKEND_URL)
        print(f"\n✅ Successfully created {len(new_tags)} new tags!")

    except Exception as e:
        print(f"❌ Failed to create tags: {e}")
        print("💡 Common issues:")
        print("   - Tag name already exists (case-sensitive)")
        print("   - API key lacks RESEARCHER permissions")
        print("   - Network connectivity problems")
        print("   - Tag name exceeds 100 character limit")
        sys.exit("Fix the issue and re-run this cell")

else:
    print("✅ No new tags needed - all tags already exist!")

print(f"\n🎯 Ready to create experiment with tags: {', '.join(tag_names_for_experiment)}")
print("✅ All tags are available in WAVE backend!")

✅ No new tags needed - all tags already exist!

🎯 Ready to create experiment with tags: test
✅ All tags are available in WAVE backend!


In [14]:
# Create test experiment in WAVE backend

print("🚀 Creating test experiment in WAVE backend...")

# Create the test experiment
test_description = f"Test experiment for {experiment_type_name} - {test_experiment_name}"
test_additional_data = {"created_by": "python_notebook"}

try:
    test_experiment = await create_experiment(
        description=test_description,
        experiment_type_id=exp_type_id,
        tags=tag_names_for_experiment,
        researcher_api_key=RESEARCHER_API_KEY,
        wave_backend_url=WAVE_BACKEND_URL,
        additional_data=test_additional_data,
    )
    experiment_id = str(test_experiment["uuid"])

    print(f"✅ Created test experiment successfully!")
    print(f"📝 Experiment Name: {test_experiment['description']}")
    print(f"🆔 Experiment ID: {experiment_id}")
    print(f"🏷️  Tags: {', '.join(test_experiment['tags'])}")
    print(f"📊 Experiment Type ID: {test_experiment['experiment_type_id']}")
    print(f"🕒 Created: {test_experiment['created_at']}")

    print(f"\n💾 Additional Data:")
    for key, value in test_experiment["additional_data"].items():
        print(f"   {key}: {value}")

except Exception as e:
    print(f"❌ Failed to create test experiment: {e}")
    print("💡 Common issues:")
    print("   - Experiment type ID doesn't exist")
    print("   - One or more tags don't exist")
    print("   - API key lacks RESEARCHER permissions")
    print("   - Network connectivity problems")
    print("   - Invalid experiment data format")
    sys.exit("Fix the issue and re-run this cell")

print(f"\n🎯 Test Experiment ID for next steps: {experiment_id}")
print(f"👤 Test Participant ID: {test_experimentee_id}")
print("✅ Ready to launch experiment with WAVE integration!")

🚀 Creating test experiment in WAVE backend...
✅ Created test experiment successfully!
📝 Experiment Name: Test experiment for jspsych_color_circles_demo - TEST_jspsych_color_circles_demo_20260101_012427
🆔 Experiment ID: 3ed5f0be-1c84-47f7-9bbd-4b7ae1a2c367
🏷️  Tags: test
📊 Experiment Type ID: 1
🕒 Created: 2026-01-01T09:26:58.646785Z

💾 Additional Data:
   created_by: python_notebook

🎯 Test Experiment ID for next steps: 3ed5f0be-1c84-47f7-9bbd-4b7ae1a2c367
👤 Test Participant ID: test_participant_8182729c
✅ Ready to launch experiment with WAVE integration!


## Phase 3: Launch Experiment with official URL, test run again!

In [15]:
# Launch experiment with WAVE integration parameters

print("🚀 Launching Experiment with WAVE Integration")
print("Now we'll open your experiment with the actual WAVE backend parameters")

# Create full URL with all required parameters
base_url = f"http://localhost:{PORT}/"
full_url, censored_url = create_experiment_url(
    base_url, experiment_id, test_experimentee_id, EXPERIMENTEE_API_KEY
)

print(f"\n🔗 Experiment URL (with hidden API key):")
print(f"   {censored_url}")
print(f"\n🔑 API Key Security:")
print(f"   Using EXPERIMENTEE key ending in: ...{EXPERIMENTEE_API_KEY[-4:]}")
print(f"   ⚠️  This key should have LIMITED permissions for data logging only")

# Open experiment in browser
print(f"\n🌐 Opening experiment in browser...")
try:
    webbrowser.open(full_url)
    print("✅ Browser opened with WAVE-integrated experiment")
    print("🔄 Server running in background - experiment should log data to WAVE now!")

except Exception as e:
    print(f"❌ Could not open browser automatically: {e}")
    print(f"\n📋 Manual steps:")
    print(f"   1. Open your browser")
    print(f"   2. Navigate to: {censored_url}")
    print(f"   3. Replace [EXPERIMENTEE_API_KEY_HIDDEN] with your actual EXPERIMENTEE API key")

print(f"\n💡 What to do next:")
print(f"   1. Complete the experiment in the browser")
print(f"   2. Verify data logging works (check browser console for any errors)")
print(f"   3. Return to this notebook to check if data was logged to WAVE")
print(f"   4. The experiment will now log data to experiment ID: {experiment_id}")

🚀 Launching Experiment with WAVE Integration
Now we'll open your experiment with the actual WAVE backend parameters

🔗 Experiment URL (with hidden API key):
   http://localhost:8080/?key=[EXPERIMENTEE_API_KEY_HIDDEN]&experiment_id=3ed5f0be-1c84-47f7-9bbd-4b7ae1a2c367&participant_id=test_participant_8182729c

🔑 API Key Security:
   Using EXPERIMENTEE key ending in: ...svBH
   ⚠️  This key should have LIMITED permissions for data logging only

🌐 Opening experiment in browser...
✅ Browser opened with WAVE-integrated experiment
🔄 Server running in background - experiment should log data to WAVE now!

💡 What to do next:
   1. Complete the experiment in the browser
   2. Verify data logging works (check browser console for any errors)
   3. Return to this notebook to check if data was logged to WAVE
   4. The experiment will now log data to experiment ID: 3ed5f0be-1c84-47f7-9bbd-4b7ae1a2c367


In [16]:
# Data validation by pulling from the WAVE backend
print("📋 Before proceeding, please confirm:")
print("1. You completed the entire experiment in the browser")
print("2. You checked the browser console for any JavaScript errors")
print("3. You verified no WAVE client error messages appeared")

completion_confirmed = get_user_confirmation(
    "\nDid you successfully complete the experiment with WAVE integration?",
    "Please complete the experiment before proceeding.",
)

print("🔍 Retrieving experiment data from WAVE backend...")
print(f"📊 Looking for data in experiment ID: {experiment_id}")

try:
    data_df = await get_experiment_data(experiment_id, RESEARCHER_API_KEY, WAVE_BACKEND_URL)

    if len(data_df) > 0:
        print(f"🎉 Success! Found {len(data_df)} data points!")
        print(f"📈 Participants: {data_df['participant_id'].nunique()}")

        # Display the data
        print(f"\n📋 Experiment Data:")
        with pd.option_context("display.max_rows", None, "display.max_columns", None):
            display(data_df)

        # Data validation
        print(f"\n🔍 Data Validation")
        print("Please review the experiment data above and confirm it matches your expectations.")
        print("\n💡 Check that:")
        print("   - All expected columns are present")
        print("   - Data types look correct (numbers, strings, booleans)")
        print("   - Values are within expected ranges")
        print("   - No missing or null values where they shouldn't be")
        print("   - Participant ID matches what you used")

        # Ask user to validate the logged data
        data_validation_passed = get_user_confirmation(
            "\nIs the logged data as expected?",
            "Please fix data issues before proceeding to production setup.\n"
            + "💡 Common issues:\n"
            + "   - Schema mismatch between processTrialData and experiment type\n"
            + "   - Missing or incorrect field names\n"
            + "   - Data type mismatches (string vs number, etc.)\n"
            + "   - JavaScript errors preventing proper data extraction\n"
            + "   - WAVE client configuration issues",
        )

        print(f"\n🎉 Excellent! Your WAVE integration is working correctly!")
        print("✅ Schema validation passed")
        print("✅ Data logging is functioning properly")

    else:
        print("⏳ No data found.")
        print("💡 Possible issues:")
        print("   - Experiment didn't complete successfully")
        print("   - JavaScript errors prevented data logging")
        print("   - WAVE client configuration problems")
        print("   - Network connectivity issues during data logging")
        get_user_confirmation(
            prompt="No data was logged, was this expected behavior? Press y to continue, n to stop and troubleshoot",
            exit_message="Unexpected failed logging -- Exiting for troubleshooting",
        )

except Exception as e:
    print(f"❌ Failed to retrieve data: {e}")
    print("💡 Common issues:")
    print("   - Experiment ID doesn't exist")
    print("   - API key lacks RESEARCHER permissions")
    print("   - Network connectivity problems")
    sys.exit("Fix the issue and re-run this cell")

print(f"\n🚀 Ready to create your production experiment!")

📋 Before proceeding, please confirm:
1. You completed the entire experiment in the browser
2. You checked the browser console for any JavaScript errors
3. You verified no WAVE client error messages appeared
Please enter your feedback to continue.


127.0.0.1 - - [01/Jan/2026 01:26:59] "GET /?key=3ZJ6E6o2BLdsSzZE4gt8svBH&experiment_id=3ed5f0be-1c84-47f7-9bbd-4b7ae1a2c367&participant_id=test_participant_8182729c HTTP/1.1" 200 -
127.0.0.1 - - [01/Jan/2026 01:28:05] code 404, message File not found
127.0.0.1 - - [01/Jan/2026 01:28:05] "GET /.well-known/appspecific/com.chrome.devtools.json HTTP/1.1" 404 -


❌ Please complete the experiment before proceeding.


SystemExit: User cancelled

## Phase 4: Production Deployment

Now that we have completed testing and verified that data is logging correctly to the WAVE client, we are ready to deploy to production.

**Next steps for production deployment:**
1. Create your production experiment (below)
2. Test the production experiment thoroughly
3. Deploy your experiment to your hosting platform
4. Distribute experiment URL (which if you did it right in the .env file, should already include the EXPERIMENTEE-level API key) to participants

Let's create your production experiment!

First, in the cell below, add some fields that characterize what makes this experiment unique

In [ ]:
# Create production experiment
# Place experimental metadata here for querying later
production_additional_data = {
    "created_by": "python_notebook",
    "production_experiment": True,
    "version_details": "<INSERT DESCRIPTION HERE>",
} # you can add additional fields as needed

print("🎯 Production Experiment Creation")
print("Now let's create your real experiment for actual data collection!")



In [ ]:
# Create production experiment

print("🎯 Production Experiment Creation")
print("Now let's create your real experiment for actual data collection!")

# Get experiment name from user
print(f"\n📝 Current experiment type: {experiment_type_name}")
print("You can use the same EXPERIMENT TYPE for multiple EXPERIMENTS, as long as it uses the same columns as you defined earlier")

experiment_name = input("\nEnter a name for your production experiment. camelCase or lower_case_with_underscores only (Press esc if you do not wish to proceed): ").strip()

if not experiment_name:
    print("❌ Experiment name cannot be empty")
    sys.exit("Please provide an experiment name")

print(f"\n🏷️  Test experiment tags: {', '.join(tag_names_for_experiment)}. (We will auto strip `test` for production)")
use_same_tags = input("Use the same tags for production experiment? We will auto strip `test`. (y/n): ").strip().lower()

if use_same_tags in ["y", "yes"]:
    production_tags = [tag for tag in tag_names_for_experiment if tag != "test"]
else:
    print("💡 Enter tag names separated by commas (or press Enter for no tags)")
    tag_input = input("Production experiment tags: ").strip()
    if tag_input:
        production_tags = [tag.strip() for tag in tag_input.split(",")]
    else:
        production_tags = []

print(f"\n🚀 Creating production experiment...")

# Create production experiment using the consolidated function
production_additional_data = {
    "created_by": "python_notebook",
    "production_experiment": True,
}

try:
    production_experiment = await create_experiment(
        description=experiment_name,
        experiment_type_id=exp_type_id,
        tags=production_tags,
        researcher_api_key=RESEARCHER_API_KEY,
        wave_backend_url=WAVE_BACKEND_URL,
        additional_data=production_additional_data,
    )
    production_experiment_id = str(production_experiment["uuid"])

    print(f"🎉 Production experiment created successfully!")
    print(f"📝 Name: {production_experiment['description']}")
    print(f"🆔 Experiment ID: {production_experiment_id}")
    print(f"🏷️  Tags: {', '.join(production_experiment['tags'])}")
    print(
        f"📊 Experiment Type: {experiment_type_name} (ID: {production_experiment['experiment_type_id']})"
    )
    print(f"🕒 Created: {production_experiment['created_at']}")

except Exception as e:
    print(f"❌ Failed to create production experiment: {e}")
    sys.exit("Fix the issue and re-run this cell")

print(f"\n🎯 Your production experiment is ready!")

## 🎊 WAVE Experiment Production Setup Complete!

**Congratulations!** Your production experiment is ready for participant data collection!

### 🔗 Your Production Experiment


### ⚠️ Security Requirements
- **Only distribute EXPERIMENTEE-level API keys** to participants (data logging only)
- **Never share your RESEARCHER API key** (full admin access)
- **Set appropriate expiration dates** on participant API keys
- **Monitor via WAVE dashboard** for unauthorized access

### 🚀 Final Deployment Process
1. **Open PR** from your feature branch to main
2. **After approval**, merge to main branch
3. **Open PR** from main → release and merge to deploy to Vercel
4. **Distribute experiment URL** with EXPERIMENTEE-level API keys to participants

Some deployment platforms like Vercel may generate a link for you to test after opening the PR to main.

### 📊 Data Collection & Monitoring
- Use the **WAVE client** to monitor incoming participant data

### 🔧 Troubleshooting
- **API Key Issues**: Verify EXPERIMENTEE vs RESEARCHER permissions
- **Connection Problems**: Ensure WAVE_BACKEND_URL matches your `params.js` file
- **Data Issues**: Re-run relevant notebook cells to regenerate components
- **Schema Mismatches**: Verify `processTrialData` function matches the schema

## 📚 Support Resources
- Experiment template documentation in `/docs`
- WAVE client documentation
- Research team for technical support

**Your experiment is live and ready for participants!** 🧪✨

## 🧹 Cleaning Up Test Experiments

**Test experiment cleanup requires admin permissions.**

Your test experiment (ID: `{experiment_id}`) and any test data created during this setup are tagged with `"test"` for easy identification.

### To clean up test experiments:

1. **Add ADMIN_API_KEY to tools/.env**
   - Get an admin-level API key with deletion permissions
   - Add it to your `tools/.env` file:
   ```
   ADMIN_API_KEY=your_admin_api_key_here
   ```

2. **Run the cleanup notebook**
   - Open `tools/cleanup_test_experiments.ipynb`
   - This will delete ALL experiments tagged with "test"
   - ⚠️ **This action cannot be undone**

### Why admin permissions are needed:
- Experiment deletion requires higher privileges than data collection
- This prevents accidental deletion during normal development
- Admin cleanup allows bulk deletion of all test experiments at once

**Important**: The cleanup notebook will delete ALL experiments with the "test" tag, not just the ones from this session.